In [1]:
import torch

In [26]:
import numpy as np

In [10]:
x = torch.empty(5,3) #construct a 5x3 matrix, uninitialised

In [14]:
x = torch.rand(5,3) #randomly initialised matrix

In [12]:
x = torch.zeros(5,3,dtype = torch.long)

In [16]:
x = torch.tensor([5.5,3])

any operation that mutates a tensor in-place is post-fixed with an _

In [17]:
print(x)

tensor([5.5000, 3.0000])


In [20]:
x = torch.randn(4,4)
y = x.view(16) #resize tensor
z = x.view(-1, 8) #the size -1 is inferred from other dimensions
print(x.size(),y.size(),z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])


In [22]:
x = torch.randn(1)
print(x)
print(x.item()) #get one-element tensor as a Python number

tensor([0.6202])
0.6201874017715454


# Numpy Bridge
The tensor and numpy array share their underlying memory locations and changing one will change the other

In [23]:
a = torch.ones(5)
print(a)
b = a.numpy() ######
print(b)

tensor([1., 1., 1., 1., 1.])
[1. 1. 1. 1. 1.]


In [24]:
a.add_(1)
print(a)
print(b)

tensor([2., 2., 2., 2., 2.])
[2. 2. 2. 2. 2.]


In [27]:
a = np.ones(5)
b = torch.from_numpy(a) ####
a += 1
print(a)
print(b)

[2. 2. 2. 2. 2.]
tensor([2., 2., 2., 2., 2.], dtype=torch.float64)


# CUDA Tensors
tensors can be moved onto any device using the .to method

In [28]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # ``.to`` can also change dtype together!

# AUTOGRAD

In [29]:
x = torch.ones(2,2,requires_grad = True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [30]:
y = x+2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [31]:
print(y.grad_fn)

In [32]:
z = y*y*3
out = z.mean()
print(z,out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward1>)


In [40]:
a = torch.randn(2,2)
a = ((a*3)/(a-1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a*a).sum()
print(b.grad_fn)

False
True


In [49]:
out.backward
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [50]:
x = torch.randn(3,requires_grad = True)
y = x*2
while y.data.norm()<1000:
    y=y*2
print(y)

tensor([ 718.1044, -138.7865, -963.2961], grad_fn=<MulBackward0>)


In [51]:
v = torch.tensor([0.1,1.0,0.0001], dtype = torch.float)
y.backward(v)
print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


In [59]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


# Neural Networks

torch.nn package, An nn.Module contains layers, and a method forward(input) that returns the output

In [60]:
import torch.nn as nn
import torch.nn.functional as F

In [75]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self,x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *=s
        return num_features
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [71]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [76]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0460,  0.1732, -0.1900, -0.1372,  0.0926,  0.0760,  0.2220, -0.0203,
         -0.0390,  0.0221]], grad_fn=<AddmmBackward>)


In [77]:
output = net(input)
target = torch.randn(10)
target = target.view(1,-1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.9543, grad_fn=<MseLossBackward>)


In [78]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

## Backprop

To backpropagate the error all we have to do is to loss.backward(). You need to clear the existing gradients though, else gradients will be accumulated to existing gradients.

In [79]:
net.zero_grad()  # zeroes the gradient buffers of all parameters

In [80]:
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([-0.0225, -0.0133, -0.0096,  0.0050, -0.0175, -0.0160])


In [82]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr = 0.01)

optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()